<h2>
Stock Market Cash Trigger</br>
by Ian Kaplan</br>
</h2>

<p>
This notebook explores a stock market timing strategy that is described in David
Alan Carter's book <i>Stock Market Cash Trigger</i>.
</p>
<p>
This Python notebook is available in the GitHub repository
<a href="https://github.com/IanLKaplan/cash_trigger">
https://github.com/IanLKaplan/cash_trigger</a>
</p>
<p>
In the <i>Stock Market Cash Trigger</i> the 200-day moving average for the S&P 500
ETF SPY close prices is compared with the current close SPY price. When the
close prices for SPY trends below the 200-day moving average, this is a
signal to move out of equity instruments into bond instruments.
</p>
<p>
Technical analysis examines movement, usually price movement, in an attempt to
discover profitable trades.  Technical analysis is sometimes compared to
voodoo magic because many of the trading signals do not have strong quantitative
evicence. Often trading based on technical analysis requires a human because the
rules are not clearly defined enough to implement in computer software.
</p>
<p>
In the <i>Stock Market Cash Trigger</i> the author, David Alan Carter,
discusses why be believes the 200-day moving average is a good trading signal
and compares it to similar signals. Rather than reproduce this discussion
here I recommend that you read the book.
</p>
<p>
The definition of "trending below" the SPY 200-day moving average is when, on the last
day of a month, the close price of SPY is below the 200-day average.
</p>
<p>
The description in the <i>Stock Market Cash Trigger</i> is designed to be used
by human traders. A human trader looking at stock market plots can often see
when the 200-day moving average for SPY is trending below the current the
SPY close price. A computer algorithm generally requires a more exact definition
(an exception might be machine learning algorithms, which are not covered in this
notebook).
</p>
<p>
If the cash trigger points to moving out of equity instruments, a
bond ETF is selected. In the <i>Stock Market Cash Trigger</i> this one
of three bond ETFS: JNK, TLT or MUB.  The
bond ETF that is chosen for the next month is the bond ETF with the
highest three month past return (this is the ETF rotation technique that is
explored at length in the notebook
<a href="https://github.com/IanLKaplan/twelve_percent/blob/master/twelve_percent.ipynb"><i>The 12% Solution</i></a>)
</p>
<p>
The ETF instruments used in this notebook are the same instruments
used in the book <i>Stock Market Cash Trigger</i>
</p>
<h3>
S&P 500 ETF
</h3>
<ul>
<li>
<p>
SPY: SPDR S&P 500 ETF Trust
</p>
<p>
SPY tracks a market-cap-weighted index of US large- and midcap stocks selected by
the S&P Committee (e.g., S&P 500).
</p>
<p>
Inception date: Jan 22, 1993
</p>
</li>
</ul>

<h3>
Bond ETFs
</h3>
<ul>
<li>
<p>
TLT: iShares 20+ Year Treasury Bond ETF
</p>
<p>
TLT tracks a market-weighted index of debt issued by the US Treasury
with remaining maturities of 20 years or more.
</p>
<p>
Inception date: 7/22/2002
</p>
</li>
<li>
<p>
JNK: SPDR Bloomberg High Yield Bond ETF
</p>
<p>
JNK tracks a market-weighted index of highly liquid, high-yield, US
dollar-denominated corporate bonds.
</p>
<p>
Inception date: 11/28/2007
</p>
</li>
<li>
<p>
MUB: iShares National Muni Bond ETF
</p>
<p>
MUB tracks a market-weighted index of investment-grade debt issued by state
and local governments and agencies. Interest is exempt from US income tax
and from AMT.
</p>
<p>
Inception date: 09/09/2007
</p>
</li>
</ul>
<h3>
Start Date
</h3>
<p>
A start date of March 3, 2008 is used to allow a sufficient lookback period
for the JNK ETF.
</p>
<h3>
Close Prices vs Adjusted Close Prices
</h3>
<p>
The SPY time series uses SPY close prices, rather than adjusted close prices
that factor in dividend payments. Although SPY does pay a small dividend
(about 1.3%), this can be ignored since the dividend is small.
</p>

In [ ]:
from datetime import datetime, timedelta
from tabulate import tabulate
from typing import List, Tuple
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.indexes.datetimes import DatetimeIndex
from dateutil.relativedelta import relativedelta
import numpy as np
from pathlib import Path
import tempfile

plt.style.use('seaborn-whitegrid')
pd.options.mode.chained_assignment = 'raise'


def get_market_data(file_name: str,
                    data_col: str,
                    symbols: List,
                    data_source: str,
                    start_date: datetime,
                    end_date: datetime) -> pd.DataFrame:
    """
      file_name: the file name in the temp directory that will be used to store the data
      data_col: the type of data - 'Adj Close', 'Close', 'High', 'Low', 'Open', Volume'
      symbols: a list of symbols to fetch data for
      data_source: yahoo, etc...
      start_date: the start date for the time series
      end_date: the end data for the time series
      Returns: a Pandas DataFrame containing the data.

      If a file of market data does not already exist in the temporary directory, fetch it from the
      data_source.
    """
    temp_root: str = tempfile.gettempdir() + '/'
    file_path: str = temp_root + file_name
    temp_file_path = Path(file_path)
    file_size = 0
    if temp_file_path.exists():
        file_size = temp_file_path.stat().st_size

    if file_size > 0:
        close_data = pd.read_csv(file_path, index_col='Date')
    else:
        panel_data: pd.DataFrame = data.DataReader(symbols, data_source, start_date, end_date)
        close_data: pd.DataFrame = panel_data[data_col]
        close_data.to_csv(file_path)
    assert len(close_data) > 0, f'Error reading data for {symbols}'
    return close_data


trading_days = 252
trading_quarter = trading_days // 4

window_size = 200

data_source = 'yahoo'

start_date_str = '2008-03-03'
start_date: datetime = datetime.fromisoformat(start_date_str)
# The "current date"
end_date: datetime = datetime.today() - timedelta(days=1)


def convert_date(some_date):
    if type(some_date) == str:
        some_date = datetime.fromisoformat(some_date)
    elif type(some_date) == np.datetime64:
        ts = (some_date - np.datetime64('1970-01-01T00:00')) / np.timedelta64(1, 's')
        some_date = datetime.utcfromtimestamp(ts)
    return some_date


def findDateIndex(date_index: DatetimeIndex, search_date: datetime) -> int:
    '''
    In a DatetimeIndex, find the index of the date that is nearest to search_date.
    This date will either be equal to search_date or the next date that is less than
    search_date
    '''
    index: int = -1
    i = 0
    search_date = convert_date(search_date)
    date_t = datetime.today()
    for i in range(0, len(date_index)):
        date_t = convert_date(date_index[i])
        if date_t >= search_date:
            break
    if date_t > search_date:
        index = i - 1
    else:
        index = i
    return index


class SpyData:
    spy_close_file = 'spy_close'
    spy_etf = "SPY"
    spy_close: pd.DataFrame
    date_index: DatetimeIndex

    def __init__(self, start_date: datetime, end_date: datetime):
        spy_start: datetime = start_date - timedelta(days=365)
        self.spy_close = get_market_data(file_name=self.spy_close_file,
                                         data_col='Close',
                                         symbols=self.spy_etf,
                                         data_source=data_source,
                                         start_date=spy_start,
                                         end_date=end_date)
        self.date_index = self.spy_close.index

    def spy_close(self, start_date: datetime, end_date: datetime) -> pd.DataFrame:
        """
        Return a section of SPY close prices from start_date to end_date
        """
        start_ix = findDateIndex(date_index=self.date_index, search_date=start_date)
        end_ix = findDateIndex(date_index=self.date_index, search_date=end_date)
        spy_close_df = pd.DataFrame()
        assert start_ix >= 0 and end_ix >= 0
        spy_close_df = self.spy_close[:][start_ix:end_ix+1].copy()
        return spy_close_df

    def avg(self, day: datetime, window: int = window_size) -> float:
        """
        :param day: the end date for the window
        :param window: the size of the window extending back from day
        :return: the average for the SPY close prices in the window
        """
        _average: float = -1.0
        end_ix = findDateIndex(date_index=self.date_index, search_date=day)
        start_ix = end_ix - (window -1)
        assert start_ix >= 0 and end_ix >= 0
        _average = self.spy_close.values[start_ix:end_ix+1].mean()
        return _average

    def running_sum(self, values: np.array, start_ix: int, end_ix: int, win_size: int) -> np.array:
            sum_l: list = []
            win_start_ix = start_ix - (win_size - 1)
            win_end_ix = start_ix
            sum = values[win_start_ix:win_end_ix + 1].sum()
            win_end_ix = win_end_ix + 1
            sum_l.append(sum)
            while win_end_ix <= end_ix:
                win_start_ix = win_end_ix - win_size
                start_val = values[win_start_ix]
                end_val = values[win_end_ix]
                sum = (sum - start_val) + end_val
                sum_l.append(sum)
                win_end_ix = win_end_ix + 1
            sum_a = np.array(sum_l)
            return sum_a

    def moving_avg(self, start_date: datetime, end_date: datetime, window: int = window_size) -> pd.DataFrame:
        """
        Compute a moving average series
        :param start_date: the start date for the moving average series
        :param end_date: the end date for the moving average series
        :param window: the window size that extends, initially, back from start_date
        :return: a moving average series as a DataFrame. The date index will be the same
                 as the SPY time DataFrame between start_date and end_date
        """
        start_ix = findDateIndex(date_index=self.date_index, search_date=start_date)
        end_ix = findDateIndex(date_index=self.date_index, search_date=end_date)
        win_start_ix = start_ix - (window - 1)
        assert start_ix >= 0 and end_ix >= 0 and win_start_ix >= 0
        sum_l: list = []
        spy_values = self.spy_close.values
        sum = spy_values[win_start_ix:start_ix + 1].sum()
        win_start_ix = win_start_ix + 1
        start_ix = start_ix + 1
        while start_ix <= end_ix:
            sum_l.append(sum)
            start_val = spy_values[win_start_ix]
            end_val = spy_values[start_ix]
            sum = (sum - start_val) + end_val
            win_start_ix = win_start_ix + 1
            start_ix = start_ix + 1



spy_data = SpyData(start_date, end_date)
spy_start_avg = spy_data.avg(start_date)


<h2>
Disclaimer
</h2>
<p>
This notebook is not financial advice, investment advice, or tax advice.
The information in this notebook is for informational and recreational purposes only.
Investment products discussed (ETFs, mutual funds, etc.) are for illustrative purposes
only. This is not a recommendation to buy, sell, or otherwise transact in any of the
products mentioned. Do your own due diligence. Past performance does not guarantee
future returns.
</p>